Trainable mixture of gaussians model where base distributions all have diagonal covariance matrix. Based on OpenAI's Baselines implementation of probability density under a multivariate gaussian 

In [1]:
import tensorflow as tf
import numpy as np

Base gaussian implementation

In [2]:
class DiagGaussianPd():
    def __init__(self, flat):
        self.flat = flat
        mean, logstd = tf.split(axis=len(flat.shape)-1, num_or_size_splits=2, value=flat)
        self.mean = mean
        self.logstd = logstd
        self.std = tf.exp(logstd)
    def flatparam(self):
        return self.flat
    def mode(self):
        return self.mean
    def neglogp(self, x):
        return 0.5 * tf.reduce_sum(tf.square((x - self.mean) / self.std), axis=-1) \
               + 0.5 * np.log(2.0 * np.pi) * tf.to_float(tf.shape(x)[-1]) \
               + tf.reduce_sum(self.logstd, axis=-1)
    def sample(self):
        return self.mean + self.std * tf.random_normal(tf.shape(self.mean))


Mixture model builds on this. It's initialised with two arguments: a list of gaussian models and a vector of mixing coefficients, defining how the base gaussians are weighted in the mixture.

In [3]:
class MixOfGaussianPd():
    def __init__(self, list_of_gaussians, coeffs): 
        self.list_of_gaussians = list_of_gaussians
        self.K = len(list_of_gaussians)
        self.coeffs = coeffs
        self.mean = tf.matmul(self.coeffs, [i.mean[0] for i in self.list_of_gaussians])
    def mode(self):
        ind = np.argmax(self.coeffs.eval()[0])
        return self.list_of_gaussians[ind].mean
    def neglogp(self, x):
        individual_ps = [tf.exp(-i.neglogp(x)) for i in self.list_of_gaussians]    
        return -tf.log(tf.matmul(self.coeffs, individual_ps))
    def sample(self):
        ind = np.random.choice(self.K, 1, p=self.coeffs.eval()[0])[0]
        return self.list_of_gaussians[ind].sample()
    def logp(self, x):
        return - self.neglogp(x)

The number of components is chosen a priori: here it's 2, arbitrarily.  Each component is described by a dictionary defining a multivariate normal distribution, with an entry for each variable. Each entry lists the corresponding mean and log standard deviation.

In [4]:
norm1 = {'x1':[0.2, np.log(0.05)], 'x2':[0.1,np.log(0.05)], 'x3':[0.9,np.log(0.02)]}
norm2 = {'x1':[0.3, np.log(0.05)], 'x2':[0.5,np.log(0.05)], 'x3':[0.7,np.log(0.03)]}

Pack the dictionaries into a list

In [5]:
list_of_normal_params = [norm1, norm2]
num_components = len(list_of_normal_params)
num_vars = len(list_of_normal_params[0])

Create the TF variables for the means and standard deviations of each component of the model and collect into a list to be used for initialising the individual gaussians

In [6]:
param_list = []

for k in range(num_components):
    mean =tf.get_variable(name="mean_%i"%k, shape=[1,num_vars], 
                          initializer=tf.constant_initializer(value=[i[0] for i in list_of_normal_params[k] \
                                                                     .values()]), trainable=True)
    logstd = tf.get_variable(name="logstd_%i"%k, shape=[1,num_vars], 
                             initializer=tf.constant_initializer(value=[i[1] for i in list_of_normal_params[k] \
                                                                        .values()]), trainable=True)
    param = tf.concat([mean, mean *0.0 +logstd], axis=1)
    param_list.append(param)



We also need mixing coefficients corresponding to a prior probability on each of the components. Initialise these to a uniform distribution

In [7]:
mix_coeffs = tf.nn.softmax(tf.get_variable(name="mix_coeffs", shape=[1,num_components],
                                           initializer=tf.constant_initializer(
                                               value=[1/num_components for i in range(num_components)]), 
                                           trainable=True))

Make a list of multivariate normals where each is initialised from a flat vector of parameters, created above

In [7]:
list_of_gaussians = [DiagGaussianPd(i) for i in param_list]

Initialise the mixture model from this list of base distributions, and the mixture coefficients

In [8]:
pd = MixOfGaussianPd(list_of_gaussians, mix_coeffs)

Trainable parameters are the vectors of means and log standard deviations, and the mixing coefficients

In [9]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'mean_0:0' shape=(1, 3) dtype=float32_ref>,
 <tf.Variable 'logstd_0:0' shape=(1, 3) dtype=float32_ref>,
 <tf.Variable 'mean_1:0' shape=(1, 3) dtype=float32_ref>,
 <tf.Variable 'logstd_1:0' shape=(1, 3) dtype=float32_ref>,
 <tf.Variable 'mix_coeffs:0' shape=(1, 2) dtype=float32_ref>]

The model implements sampling and log probability. It can be trained in Tensorflow by defining a loss that includes the pd.logp() function, and computing derivatives with respect to its parameters

In [10]:
sess= tf.InteractiveSession()
tf.get_default_session().run(tf.global_variables_initializer())

In [11]:
pd.sample().eval()

array([[ 0.30372337,  0.63920331,  0.73118794]], dtype=float32)

In [12]:
pd.logp([0.7, 0.1, 0.6]).eval()

array([[-63.50749207]], dtype=float32)

In [14]:
pd.mean.eval()

array([[ 0.25      ,  0.30000001,  0.79999995]], dtype=float32)